In [6]:
# pip install fredapi pandas numpy
import os
import numpy as np
import pandas as pd
from fredapi import Fred

# -------- configuration --------
FRED_KEY = os.getenv("FRED_API_KEY") or "PUT_YOUR_FRED_KEY_HERE"
fred = Fred(api_key=FRED_KEY)

start = "2024-01-01"  # adjust as needed
end = "2025-06-30"  # None = latest available

# FRED daily series and transforms
series_defs = {
    "CPIAUCSL": {"transform": "logret"},  # Consumer Price Index for All Urban Consumers: All Items
}


def fetch_fred_series(sid, start=None, end=None):
    s = fred.get_series(sid, observation_start=start, observation_end=end)
    s.name = sid
    return s.sort_index()


def transform(s: pd.Series, method: str) -> pd.Series:
    if method == "logret":
        return np.log(s).diff()
    if method == "deltalevel":
        return s.diff()
    # "level"
    return s


# -------- download and transform --------
features = []
for sid, meta in series_defs.items():
    raw = fetch_fred_series(sid, start, end).dropna()
    feat = transform(raw, meta["transform"]).rename(sid)
    features.append(feat)

X = pd.concat(features, axis=1)

# Optional: forward-fill short gaps due to holidays or missing days
X = X.ffill(limit=5)

# Drop initial NaNs created by differences/log returns
X = X.dropna()

# -------- save --------
X.to_csv("data/fred/fred_daily_features.csv")
print("Saved fred_daily_features.csv with shape:", X.shape)
print(X.tail())


Saved fred_daily_features.csv with shape: (17, 1)
            CPIAUCSL
2025-02-01  0.002157
2025-03-01 -0.000500
2025-04-01  0.002206
2025-05-01  0.000808
2025-06-01  0.002866
